# Import Bibliotheken

In [1]:
# Festlegung des Device
import platform

# Laden der Daten
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Operationen mit Texten
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import spacy

# Modell
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

# Evaluierung
import torchmetrics

# Auswertung der Evaluation
import pandas as pd
import altair as alt
import numpy as np

from sklearn import preprocessing

c:\Users\loedu\anaconda3\envs\ai_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Package Versionen

In [2]:
for package in [torchtext,torch, torchmetrics, spacy, alt, pd]:
    print(f'{package.__name__}: {package.__version__}')

torchtext: 0.13.1
torch: 1.12.1
torchmetrics: 0.9.3
spacy: 3.4.2
altair: 4.2.0
pandas: 1.4.3


# Device Auswahl

In [3]:
device = torch.device(
    "cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Daten

## Dataset

### Erstellung Dataset

In [4]:
comment_df = pd.read_excel('../../data/clean/Google_Rezensionen.xlsx')
comment_df

,Unnamed: 0,Tankstellenname,Datum,Bewertung,Kommentar,Kategorien
0,4,TS Schleswig MC,2022-09-30 06:46:42,5,super nettes personal gutes frühstück mit groß...,"Personal,Shop,Bistro"
1,5,TS Prisdorf,2022-09-29 17:22:44,5,sehr höfliche bedienstete,Personal
2,6,TS Wanderup,2022-09-29 15:38:02,5,immer gerne dort nettes und zuvorkommendes per...,"Personal,Waschanlage"
3,10,TS Handewitt,2022-09-28 17:05:28,5,nettes freundliches personal,Personal
4,12,TS Bremen,2022-09-28 04:24:34,4,normale tankstelle die aber in der tank app ni...,"Pricing,DigitalFueling"
...,...,...,...,...,...,...
3992,11072,TS Jübek,2015-04-07 17:45:02,5,dies ist einfach die beste tankstelle die ich ...,"Kraftstoffauswahl,Waschanlage"
3993,11077,TS Jübek,2014-07-16 07:08:37,5,sehr gut günstig guter shop und einige andere ...,"Pricing,Shop,Kraftstoffauswahl,Waschanlage,Sta..."
3994,11079,TS Neustadt am Rüb.,2013-09-21 15:26:28,5,öffnungszeiten mo so 06 00 22 00 uhr,Öffnungszeiten
3995,11080,TS Handewitt(SP),2012-11-05 11:16:00,5,genug platz fuer wohnwagen rechts beim autogas...,"Personal,Kraftstoffauswahl"


In [5]:
class TeamGoogleBewertungenDataSet(Dataset):

    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = comment_df.iloc[index]['Kommentar']
        category = comment_df.iloc[index]['Kategorien']
        return text, category

### Bestimmung Test- und Traningsdaten

In [6]:
# Bestimmung der Länge der Test- und Traningsdaten
dataset = TeamGoogleBewertungenDataSet(comment_df)
len_train = round(len(dataset)* 0.7)
len_test = round(len(dataset) * 0.3)
assert len(comment_df) == len_train + len_test

In [7]:
# Zufällige Aufsplittung der Anzahl von Test- und Trainingsdaten
train_set, test_set = torch.utils.data.random_split(dataset, [len_train, len_test]) 

In [8]:
# next(iter(train_set))

## Dataloader

### Tokenisierung

In [9]:
tokenizer = get_tokenizer(
    'spacy', 
    language='de_core_news_lg')

In [10]:
# Beispiel für Anwendung des tokenizer
# sentence = "This isn't a very long example."
# tokenizer(sentence)

### Erstellung Vokabular

In [11]:
def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)

In [12]:
# Länge des Trainungsets
len(train_set)

2798

In [13]:
tokens = yield_tokens(train_set)

In [14]:
next(iter(tokens))

['nettes',
 'personal',
 'beim',
 'waschgang',
 'gründliche',
 'vorwäsche',
 'von',
 'hand']

In [15]:
vocab = build_vocab_from_iterator(tokens, min_freq=2, specials=["<unk>"])

vocab.set_default_index(vocab["<unk>"])

In [16]:
len(vocab)

2338

In [17]:
vocab(['Hallo', 'sehr', 'Personal'])

[0, 3, 0]

In [18]:
vocab.lookup_token(0)

'<unk>'

### Encoding

In [19]:
onehot = torch.zeros(1,len(vocab))

In [20]:
pos = vocab(['nette', 'Bedienung', 'sauber'])
pos

[30, 0, 44]

In [21]:
def encode(text, vocab):
    tokens = tokenizer(text)
    onehot = torch.zeros(1,len(vocab))
    onehot[:,vocab(tokens)] = 1
    return onehot

### Multilabel Encoding

In [22]:
import torch
import torch.nn

In [23]:
df = comment_df.explode('Kategorien')
df

,Unnamed: 0,Tankstellenname,Datum,Bewertung,Kommentar,Kategorien
0,4,TS Schleswig MC,2022-09-30 06:46:42,5,super nettes personal gutes frühstück mit groß...,"Personal,Shop,Bistro"
1,5,TS Prisdorf,2022-09-29 17:22:44,5,sehr höfliche bedienstete,Personal
2,6,TS Wanderup,2022-09-29 15:38:02,5,immer gerne dort nettes und zuvorkommendes per...,"Personal,Waschanlage"
3,10,TS Handewitt,2022-09-28 17:05:28,5,nettes freundliches personal,Personal
4,12,TS Bremen,2022-09-28 04:24:34,4,normale tankstelle die aber in der tank app ni...,"Pricing,DigitalFueling"
...,...,...,...,...,...,...
3992,11072,TS Jübek,2015-04-07 17:45:02,5,dies ist einfach die beste tankstelle die ich ...,"Kraftstoffauswahl,Waschanlage"
3993,11077,TS Jübek,2014-07-16 07:08:37,5,sehr gut günstig guter shop und einige andere ...,"Pricing,Shop,Kraftstoffauswahl,Waschanlage,Sta..."
3994,11079,TS Neustadt am Rüb.,2013-09-21 15:26:28,5,öffnungszeiten mo so 06 00 22 00 uhr,Öffnungszeiten
3995,11080,TS Handewitt(SP),2012-11-05 11:16:00,5,genug platz fuer wohnwagen rechts beim autogas...,"Personal,Kraftstoffauswahl"


#### Kategorien splitten

In [24]:
einzelLabels = comment_df['Kategorien'].str.split(',')
label_list_export = einzelLabels.to_list()
label_list_export

[['Personal', 'Shop', 'Bistro'],
 ['Personal'],
 ['Personal', 'Waschanlage'],
 ['Personal'],
 ['Pricing', 'DigitalFueling'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal', 'Pricing'],
 ['Personal', 'Pricing', 'Kraftstoffauswahl'],
 ['Personal', 'Öffnungszeiten'],
 ['Personal', 'SB-Waschboxen', 'Waschanlage'],
 ['Shop'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal', 'Waschanlage'],
 ['Personal', 'Pricing'],
 ['Personal', 'Pricing'],
 ['Bistro'],
 ['Personal', 'Pricing', 'Shop', 'Waschanlage'],
 ['Personal'],
 ['Personal'],
 ['Bistro'],
 ['Erscheinungsbild'],
 ['Personal', 'Pricing'],
 ['Personal'],
 ['Personal'],
 ['Personal', 'Shop'],
 ['Erscheinungsbild'],
 ['Personal', 'Bistro'],
 ['Erscheinungsbild', 'Nacht-/Tankautomat', 'Waschanlage', 'Staubsauger'],
 ['Pricing'],
 ['Personal'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal'],
 ['Bistro'],
 ['Pricing', 'Kraftstoffauswahl'],
 ['Pricing'],
 ['Nacht-/Tankautomat'],
 ['Shop', 'Bistro'],
 ['Erscheinungsbild', 'Personal', 'Sanitär

##### In der nachfolgenden Zelle wird aus der verschachtelten Liste eine "flache Liste"

In [25]:
flat_list = []
for sublist in label_list_export:
    for item in sublist:
        flat_list.append(item)

flat_list

['Personal',
 'Shop',
 'Bistro',
 'Personal',
 'Personal',
 'Waschanlage',
 'Personal',
 'Pricing',
 'DigitalFueling',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Pricing',
 'Personal',
 'Pricing',
 'Kraftstoffauswahl',
 'Personal',
 'Öffnungszeiten',
 'Personal',
 'SB-Waschboxen',
 'Waschanlage',
 'Shop',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Waschanlage',
 'Personal',
 'Pricing',
 'Personal',
 'Pricing',
 'Bistro',
 'Personal',
 'Pricing',
 'Shop',
 'Waschanlage',
 'Personal',
 'Personal',
 'Bistro',
 'Erscheinungsbild',
 'Personal',
 'Pricing',
 'Personal',
 'Personal',
 'Personal',
 'Shop',
 'Erscheinungsbild',
 'Personal',
 'Bistro',
 'Erscheinungsbild',
 'Nacht-/Tankautomat',
 'Waschanlage',
 'Staubsauger',
 'Pricing',
 'Personal',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Bistro',
 'Pricing',
 'Kraftstoffauswahl',
 'Pricing',
 'Nacht-/Tankautomat',
 'Shop',
 'Bistro',
 'Erscheinungsbild',
 'Personal',
 'Sanitär',
 'Bistro',
 'Pricing',
 'Verkehrsanbindung'

##### Doppelte Werte werden entfernt, so dass jeder Eintrag nur einmalig vorhanden ist. Dies erreichen wir mit set()

In [26]:
flat_set = set(flat_list)
flat_set

{'AdBlue',
 'Bistro',
 'DigitalFueling',
 'E-Fuels',
 'E-Mobilität',
 'Erscheinungsbild',
 'Kartenakzeptanzen',
 'Kraftstoffauswahl',
 'Luft',
 'Nacht-/Tankautomat',
 'Paketservice',
 'Personal',
 'Pricing',
 'SB-Waschboxen',
 'Sanitär',
 'Shop',
 'Staubsauger',
 'Tankpool',
 'Verkehrsanbindung',
 'WLAN',
 'Waschanlage',
 'Werkstatt',
 'Zapfsäulen',
 'Öffnungszeiten'}

In [27]:
unflat = [[x] for x in flat_set]

In [28]:
label_vocab = build_vocab_from_iterator(unflat, min_freq=1, specials=["<unk>"])

label_vocab.set_default_index(vocab["<unk>"])

In [29]:
label_vocab.lookup_token(3)

'DigitalFueling'

#### Dictionary erstellen

In [30]:
label_dict = {label:i for i, label in enumerate(flat_set)}
label_dict

{'Werkstatt': 0,
 'Sanitär': 1,
 'E-Fuels': 2,
 'Shop': 3,
 'Kraftstoffauswahl': 4,
 'Kartenakzeptanzen': 5,
 'E-Mobilität': 6,
 'WLAN': 7,
 'Öffnungszeiten': 8,
 'Staubsauger': 9,
 'Pricing': 10,
 'SB-Waschboxen': 11,
 'Paketservice': 12,
 'DigitalFueling': 13,
 'Personal': 14,
 'Verkehrsanbindung': 15,
 'Luft': 16,
 'Erscheinungsbild': 17,
 'Bistro': 18,
 'AdBlue': 19,
 'Zapfsäulen': 20,
 'Tankpool': 21,
 'Waschanlage': 22,
 'Nacht-/Tankautomat': 23}

#### Tensor

In [31]:
doc2 = label_list_export[2]
doc2

['Personal', 'Waschanlage']

In [32]:
pos_vals = [label_dict[val] for val in doc2]
pos_vals

[14, 22]

In [33]:
labels = torch.LongTensor(pos_vals)
labels

tensor([14, 22])

In [34]:
y_onehot = nn.functional.one_hot(labels, num_classes=24)
y_onehot

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]])

In [35]:
y_onehot = y_onehot.sum(dim=0).float()
y_onehot

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 1., 0.])

In [36]:
# labels = torch.tensor([1,5,8,0])
# labels = labels.unsqueeze(0)
# target = torch.zeros(labels.size(0), 24).scatter_(1, labels, 1.)
# print(target)

### Erstellung Dataloader

In [37]:
# label_pipeline = lambda x: int(x) -1

In [38]:
def collate_batch(batch):
    label_list, text_list = [], []
 
    for (_text,_label) in batch:
    
        # Vorverarbeitung der Label
        # label_list.append(label_pipeline(_label))
        processed_labels = encode(_label, label_vocab)

        label_list.append(processed_labels)

        # Vorverarbeitung der Texte
        processed_text = encode(_text, vocab)
        
        # Zusammenführen sämtlicher Textrepräsentationen in einer Liste
        text_list.append(processed_text)
 
    # Zusammenführen aller Label in einem Tensor
    labels = torch.cat(label_list, dim=0)
    
    # Verbinden der Tensoren in text_list zu einem Tensor
    texts = torch.cat(text_list, dim = 0)

    # Ausgabe der Texte und der Label
    return texts.to(device), labels.to(device)

In [39]:
train_loader = DataLoader(
 train_set, batch_size=8,
 shuffle=True,
 collate_fn=collate_batch,
 num_workers=0
)

## Architektur und Training

### Architektur

In [40]:
class LinearTextClassificationModel(nn.Module):

    def __init__(self, vocab_size, num_class):
        super(LinearTextClassificationModel, self).__init__()
        self.fc1 = nn.Linear(vocab_size, 200)
        self.fc2 = nn.Linear(200, num_class)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.softmax(x, dim=1)
        return output

In [41]:
vocab_size = len(vocab)
num_class = 25
model = LinearTextClassificationModel(vocab_size, num_class).to(device)

### Training

In [42]:
# Hyperparameter

## Festlegung Lernrate
learning_rate = 0.1

## Initialisierung Fehlerfunktion
loss_fn = nn.MSELoss()

## Initialisierung Optimizer
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate, momentum=0.9)

## Definition der Epochen
num_epochs = 250

In [43]:
train_accuracy = torchmetrics.Accuracy(subset_accuracy=True,threshold=0.5).to(device)

loss_hist = {}
accuracy_hist = {}

In [44]:
for (text, label) in train_loader:
    #print(label[[0][0]])
    #print(text)
    labelnew = label.type(torch.LongTensor)
    print(labelnew[[0][0]])

tensor([1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        0])
tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,


In [45]:
# Training
model.train()

for epoch in range(num_epochs):

    # Dokumentation Loss -> Erkennung ob Netz konvergiert
    running_loss = 0.0
    num_batches = 0

    for (text, label) in train_loader:
        num_batches += 1
        pred = model(text)

        # Der Fehler wird berechnet
        loss = loss_fn(pred, label)
        # Der Fehler wird über das Netz zurückpropagiert
        loss.backward()
        # Die Gewichte werden angepasst
        optimizer.step()
        # Gradienten zurücksetzen
        optimizer.zero_grad()
        ## Bestimmung der Accuracy für den Batch
        train_accuracy(pred, label.type(torch.LongTensor))
        
        # running loss
        running_loss +=loss.item()
 
    loss_hist[epoch] = running_loss/num_batches
 
    batch_train_accuracy = train_accuracy.compute()
    print(f"Training Accuracy for epoch {epoch}:{batch_train_accuracy}")
    accuracy_hist[epoch] = batch_train_accuracy.cpu().numpy()
    train_accuracy.reset()

Training Accuracy for epoch 0:0.0007147963042370975
Training Accuracy for epoch 1:0.20228734612464905
Training Accuracy for epoch 2:0.29664045572280884
Training Accuracy for epoch 3:0.3048606216907501
Training Accuracy for epoch 4:0.30771979689598083
Training Accuracy for epoch 5:0.3080771863460541
Training Accuracy for epoch 6:0.3120085895061493
Training Accuracy for epoch 7:0.3166547417640686
Training Accuracy for epoch 8:0.34667620062828064
Training Accuracy for epoch 9:0.360972136259079
Training Accuracy for epoch 10:0.36990708112716675
Training Accuracy for epoch 11:0.3724088668823242
Training Accuracy for epoch 12:0.3741958439350128
Training Accuracy for epoch 13:0.3784846365451813
Training Accuracy for epoch 14:0.37919941544532776
Training Accuracy for epoch 15:0.38098642230033875
Training Accuracy for epoch 16:0.381343811750412
Training Accuracy for epoch 17:0.3824160099029541
Training Accuracy for epoch 18:0.3824160099029541
Training Accuracy for epoch 19:0.3834882080554962
Tr

In [46]:
total_train_accuracy = train_accuracy.compute()

c:\Users\loedu\anaconda3\envs\ai_env\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: The ``compute`` method of metric Accuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)


### Auswertung Fehlerentwicklung

In [47]:
loss_df = pd.DataFrame.from_dict(loss_hist, orient= 'index').reset_index()

loss_df.columns = ['Epoch', 'Loss']
loss_df.head()

,Epoch,Loss
0,0,0.082807
1,1,0.063927
2,2,0.060709
3,3,0.059371
4,4,0.058417


In [48]:
train_chart = alt.Chart(loss_df).mark_line().encode(
    x=alt.X('Epoch', title = 'Anzahl Epochen'),
    y=alt.Y('Loss', title = 'Mittlerer Fehler')
)
# glue('train-loss-team', train_chart,display=True)
train_chart

alt.Chart(...)

### Auswertung Genauigkeit

In [49]:
accuracy_df = pd.DataFrame.from_dict(accuracy_hist, orient = 'index').reset_index()
accuracy_df.columns = ['Epoch', 'Accuracy']
accuracy_df.head()

,Epoch,Accuracy
0,0,0.000715
1,1,0.202287
2,2,0.296640
3,3,0.304861
4,4,0.307720


In [50]:
accuracy_chart = alt.Chart(accuracy_df).mark_line().encode(
    x=alt.X('Epoch',title = 'Anzahl Epochen'),
    y=alt.Y('Accuracy', title = 'Genauigkeit')
)
accuracy_chart

alt.Chart(...)

## Evaluierung

In [51]:
model.eval()

LinearTextClassificationModel(
  (fc1): Linear(in_features=2338, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=25, bias=True)
)

### Gesamtevaluierung

In [52]:
test_loader = DataLoader(test_set, batch_size= 8, shuffle=True,collate_fn=collate_batch)

In [53]:
valid_accuracy = torchmetrics.Accuracy(subset_accuracy=True,threshold=0.5).to(device)

with torch.no_grad():
    for (text, label) in test_loader:
        # Vorhersage wird für das Model erzeugt
        pred = model(text)
        valid_accuracy.update(pred, label.type(torch.LongTensor))
    total_valid_accuracy = valid_accuracy.compute()

In [54]:
total_valid_accuracy

tensor(0.4404)

### Evaluierung nach Klassen

In [55]:
# Evaluieren
valid_accuracy = torchmetrics.Accuracy(
    average=None,
    num_classes=25
).to(device)

with torch.no_grad():
    for(text, label) in test_loader:
        # Die Vorhersage wird für das Model erzeugt
        pred = model(text)
        valid_accuracy.update(pred, label.type(torch.LongTensor))
    total_valid_accuracy = valid_accuracy.compute()
    total_valid_accuracy

In [56]:
total_valid_accuracy

tensor([0.6222, 0.9925, 0.8899, 0.9917, 0.9992, 0.9975, 0.8916, 0.9650, 0.9399,
        0.9758, 0.9758, 0.9917, 0.7064, 0.8624, 0.9900, 0.9600, 0.8732, 0.9908,
        0.9983, 0.9766, 0.9992, 0.9233, 0.9917, 0.9842, 0.9750])

### Evaluierung nach der Anzahl der Epochen

In [57]:
train_loader = DataLoader(train_set, batch_size=8, shuffle=True, collate_fn=collate_batch)

In [58]:
test_loader = DataLoader(test_set, batch_size=8, shuffle=True, collate_fn=collate_batch)

In [59]:
vocab_size = len(vocab)
num_class = 25
model = LinearTextClassificationModel(vocab_size, num_class).to(device)

In [60]:
learning_rate = 0.1

loss_fn = nn.MSELoss()

optimizer = optim.SGD(model.parameters(),lr=learning_rate,momentum=0.9)

num_epochs = 250

In [61]:
# Evaluierung wurd vorbereitet
train_accuracy = torchmetrics.Accuracy(subset_accuracy=True,threshold=0.5).to(device)
valid_accuracy = torchmetrics.Accuracy(subset_accuracy=True,threshold=0.5).to(device)

train_accuracy_hist = {}
train_loss_hist = {}

valid_accuracy_hist = {}
valid_loss_hist = {}

In [62]:
for epoch in range(num_epochs):

    # Loss wird dokumentiert, sodass zu erkenne ist, ob das Netz konvergiert
    num_train_batches = 0
    num_eval_batches = 0
    train_running_loss = 0.0
    eval_running_loss = 0.0

    model.train()
    for (text, label) in train_loader:
        num_train_batches += 1
        # Die Vorhersage wird für das Model erzeugt
        pred = model(text)
        # Der Fehler wird berechnet
        # Als Ausgabe wird ein Tensor zurückgegeben
        loss = loss_fn(pred, label)
        # Der Fehler wird über das Netz zurükpropagiert
        loss.backward()
        # Die Gewichte werden angepasst
        optimizer.step()
        # Gradienten zurücksetzen
        optimizer.zero_grad()
        # Aktualisierung des States der train_accurracy
        train_accuracy.update(pred, label.type(torch.LongTensor))
        # running loss
        train_running_loss += loss.item()

    train_loss_hist[epoch] = train_running_loss / num_train_batches

    # Berechnung der Trainingsgenauigkeit für die Epoche
    total_train_accuracy = train_accuracy.compute()
    # Hinzufügen der Trainingsgenauigkeit zu dem Dictionary
    train_accuracy_hist[epoch] = total_train_accuracy.cpu().numpy()

    # Validierung
    model.eval()
    with torch.no_grad():
        for (text, label) in test_loader:
            num_eval_batches += 1
            # Die Vorhersage wird für das Model erzeugt
            pred = model(text)
            # Test loss
            eval_loss = loss_fn(pred, label)
            # Running loss
            eval_running_loss += eval_loss.item()
            # Aktualisierung des States der valid_accuracy
            valid_accuracy.update(pred, label.type(torch.LongTensor))
    
    valid_loss_hist[epoch] = eval_running_loss/num_eval_batches

    # Berechnung der Validierungsgenauigkeit für die Epoche
    total_valid_accuracy = valid_accuracy.compute()
    # Hinzufügen der Validierungsgenauigkeit zum Dictionary
    valid_accuracy_hist[epoch] = total_valid_accuracy.cpu().numpy()

    print(f"Training Accuracy for epoch {epoch}: {total_train_accuracy}")
    print(f"Validation Accuracy for epoch {epoch}: {total_valid_accuracy}")
    train_accuracy.reset()
    valid_accuracy.reset()

Training Accuracy for epoch 0: 0.0007147963042370975
Validation Accuracy for epoch 0: 0.030859049409627914
Training Accuracy for epoch 1: 0.1990707665681839
Validation Accuracy for epoch 1: 0.2702251970767975
Training Accuracy for epoch 2: 0.29878485202789307
Validation Accuracy for epoch 2: 0.30025020241737366
Training Accuracy for epoch 3: 0.3055754005908966
Validation Accuracy for epoch 3: 0.3044203519821167
Training Accuracy for epoch 4: 0.3066475987434387
Validation Accuracy for epoch 4: 0.3077564537525177
Training Accuracy for epoch 5: 0.3084346055984497
Validation Accuracy for epoch 5: 0.3010842502117157
Training Accuracy for epoch 6: 0.31236597895622253
Validation Accuracy for epoch 6: 0.31025853753089905
Training Accuracy for epoch 7: 0.31415295600891113
Validation Accuracy for epoch 7: 0.3177648186683655
Training Accuracy for epoch 8: 0.3406004309654236
Validation Accuracy for epoch 8: 0.34278565645217896
Training Accuracy for epoch 9: 0.35954251885414124
Validation Accuracy 

In [63]:
train_loss_df = pd.DataFrame.from_dict(train_loss_hist, orient = "index").reset_index()
train_loss_df["type"] = "Train"
valid_loss_df = pd.DataFrame.from_dict(valid_loss_hist, orient = "index").reset_index()
valid_loss_df["type"] = "Test"
loss_df = pd.concat([train_loss_df,valid_loss_df])
loss_df.columns = ["Epoch", "Loss", "Type"]

In [64]:
train_eval_loss_chart = alt.Chart(loss_df).mark_line().encode(
    x = alt.X('Epoch', title='Anzahl Epochen'),
    y = alt.Y('Loss',title='Mittlerer Fehler'),
    color = alt.Color('Type',title='Typ')
)
train_eval_loss_chart

alt.Chart(...)

In [65]:
valid_accuracy_hist

{0: array(0.03085905, dtype=float32),
 1: array(0.2702252, dtype=float32),
 2: array(0.3002502, dtype=float32),
 3: array(0.30442035, dtype=float32),
 4: array(0.30775645, dtype=float32),
 5: array(0.30108425, dtype=float32),
 6: array(0.31025854, dtype=float32),
 7: array(0.31776482, dtype=float32),
 8: array(0.34278566, dtype=float32),
 9: array(0.34862384, dtype=float32),
 10: array(0.35446206, dtype=float32),
 11: array(0.35946622, dtype=float32),
 12: array(0.3644704, dtype=float32),
 13: array(0.36363637, dtype=float32),
 14: array(0.36697248, dtype=float32),
 15: array(0.3644704, dtype=float32),
 16: array(0.36864054, dtype=float32),
 17: array(0.36864054, dtype=float32),
 18: array(0.3678065, dtype=float32),
 19: array(0.36947456, dtype=float32),
 20: array(0.3678065, dtype=float32),
 21: array(0.37030858, dtype=float32),
 22: array(0.37030858, dtype=float32),
 23: array(0.36947456, dtype=float32),
 24: array(0.37281066, dtype=float32),
 25: array(0.37281066, dtype=float32),
 2

In [66]:
train_accuracy_hist

{0: array(0.0007148, dtype=float32),
 1: array(0.19907077, dtype=float32),
 2: array(0.29878485, dtype=float32),
 3: array(0.3055754, dtype=float32),
 4: array(0.3066476, dtype=float32),
 5: array(0.3084346, dtype=float32),
 6: array(0.31236598, dtype=float32),
 7: array(0.31415296, dtype=float32),
 8: array(0.34060043, dtype=float32),
 9: array(0.35954252, dtype=float32),
 10: array(0.36812007, dtype=float32),
 11: array(0.37312365, dtype=float32),
 12: array(0.37526804, dtype=float32),
 13: array(0.37848464, dtype=float32),
 14: array(0.37919942, dtype=float32),
 15: array(0.38098642, dtype=float32),
 16: array(0.3813438, dtype=float32),
 17: array(0.38313082, dtype=float32),
 18: array(0.38313082, dtype=float32),
 19: array(0.38420302, dtype=float32),
 20: array(0.3838456, dtype=float32),
 21: array(0.3838456, dtype=float32),
 22: array(0.3838456, dtype=float32),
 23: array(0.38420302, dtype=float32),
 24: array(0.38420302, dtype=float32),
 25: array(0.3834882, dtype=float32),
 26: 

In [67]:
# Erste Epoche löschen, da diese nicht neu initialisiert wird und Daten aus vorherigen Schritten enthält

del train_accuracy_hist[0]
del valid_accuracy_hist[0]

In [68]:
train_accuracy_df = pd.DataFrame.from_dict(train_accuracy_hist, orient = "index").reset_index()
train_accuracy_df["type"] = "Train"
valid_accuracy_df = pd.DataFrame.from_dict(valid_accuracy_hist, orient = "index").reset_index()
valid_accuracy_df["type"] = "Test"
accuracy_df = pd.concat([train_accuracy_df,valid_accuracy_df])
accuracy_df.columns = ["Epoch", "Accuracy", "Type"]

In [69]:
accuracy_df

,Epoch,Accuracy,Type
0,1,0.199071,Train
1,2,0.298785,Train
2,3,0.305575,Train
3,4,0.306648,Train
4,5,0.308435,Train
...,...,...,...
244,245,0.437865,Test
245,246,0.439533,Test
246,247,0.438699,Test
247,248,0.439533,Test


In [70]:
train_eval_accuracy_chart = alt.Chart(accuracy_df).mark_line().encode(
    x = alt.X('Epoch',title='Anzahl Epochen'),
    y = alt.Y('Accuracy',title='Genauigkeit'),
    color = alt.Color('Type',title='Typ')
)
train_eval_accuracy_chart

alt.Chart(...)